# Restaurants in the 'Coolest' Neighborhoods in Miami

In [144]:

from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('expand_frame_repr', False)

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

import requests # library to handle requests

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [145]:
content = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami').text
parser = BeautifulSoup(content, "lxml")

In [139]:
#print(parser.prettify())

### Get rows from table

In [146]:
table = parser.find(lambda tag: tag.name=='table')

rowList = []
 
rows = table.select('tr')

for tr in rows:
    td = tr.findAll(lambda tag: tag.name=='td')
    if(len(td) > 0):
        row = [tr.text.replace('\n', ' ').strip() for tr in td]
    else:
        th = rows[0].findAll(lambda tag: tag.name=='th')
        row = [tr.text.replace('\n', ' ').strip() for tr in th]
       
    rowList.append(row)

In [141]:
#print(rowList)

### Convert to dataframe and cleanup

In [147]:
df = pd.DataFrame(rowList)
selected_columns = df[[0,5]]
df = selected_columns.copy()
df.columns = ['Neighborhood', 'Coordinates']
df = df.drop(df.index[0])
print (df.to_string(index = False))

                  Neighborhood     Coordinates
                    Allapattah  25.815,-80.224
 Arts & Entertainment District  25.799,-80.190
                      Brickell  25.758,-80.193
                   Buena Vista  25.813,-80.192
                 Coconut Grove  25.712,-80.257
                     Coral Way  25.750,-80.283
               Design District  25.813,-80.193
                      Downtown  25.774,-80.193
                     Edgewater  25.802,-80.190
                       Flagami  25.762,-80.316
             Grapeland Heights  25.792,-80.258
               Health District                
                  Liberty City  25.832,-80.225
                  Little Haiti  25.824,-80.191
                 Little Havana  25.773,-80.215
                   Lummus Park  25.777,-80.201
                       Midtown  25.807,-80.193
                      Overtown  25.787,-80.201
                     Park West  25.785,-80.193
                     The Roads  25.756,-80.207
             

### Get only 'cool' neighborhoods

In [148]:
df = df.loc[df['Neighborhood'].isin(['Arts & Entertainment District','Brickell','Coconut Grove','Coral Way',
                                     'Design District','Downtown','Little Havana','Midtown','Wynwood'])]
print(df.to_string(index = False))

                  Neighborhood     Coordinates
 Arts & Entertainment District  25.799,-80.190
                      Brickell  25.758,-80.193
                 Coconut Grove  25.712,-80.257
                     Coral Way  25.750,-80.283
               Design District  25.813,-80.193
                      Downtown  25.774,-80.193
                 Little Havana  25.773,-80.215
                       Midtown  25.807,-80.193
                       Wynwood  25.804,-80.199


### Break out Latitude and Longitude from Coordinates

In [149]:
df['Latitude'] = df['Coordinates'].str[0:6].astype(float)
df['Longitude'] = df['Coordinates'].str[7:13].astype(float)
del df['Coordinates']
print (df)

                     Neighborhood  Latitude  Longitude
2   Arts & Entertainment District    25.799     -80.19
3                        Brickell    25.758     -80.19
5                   Coconut Grove    25.712     -80.25
6                       Coral Way    25.750     -80.28
7                 Design District    25.813     -80.19
8                        Downtown    25.774     -80.19
15                  Little Havana    25.773     -80.21
17                        Midtown    25.807     -80.19
25                        Wynwood    25.804     -80.19


### Use geopy library to get the latitude and longitude values for Miami Beach, FL

In [150]:
address = 'Miami Beach, FL'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Miami Beach, FL are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Miami Beach, FL are 25.7929198, -80.1353006.


### Add Miami Beach neighborhood which is missing in the dataframe

In [151]:
df = df.append({'Neighborhood' : 'Miami Beach' , 'Latitude' : 25.79, 'Longitude' : -80.13 } , ignore_index=True)
print(df)

                    Neighborhood  Latitude  Longitude
0  Arts & Entertainment District    25.799     -80.19
1                       Brickell    25.758     -80.19
2                  Coconut Grove    25.712     -80.25
3                      Coral Way    25.750     -80.28
4                Design District    25.813     -80.19
5                       Downtown    25.774     -80.19
6                  Little Havana    25.773     -80.21
7                        Midtown    25.807     -80.19
8                        Wynwood    25.804     -80.19
9                    Miami Beach    25.790     -80.13


In [152]:
# create map of Miami using latitude and longitude values
map_miami = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], 
                                  df['Longitude'], 
                                  df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  

display(map_miami) 

In [172]:
CLIENT_ID = '' #Foursquare ID
CLIENT_SECRET = '' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Get the venues for the neighborhoods.

In [154]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [155]:
miami_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Arts & Entertainment District
Brickell
Coconut Grove
Coral Way
Design District
Downtown
Little Havana
Midtown
Wynwood
Miami Beach


In [157]:
print(miami_venues.shape)
miami_venues.head()

(322, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arts & Entertainment District,25.799,-80.19,Ted's at YoungArts,25.797405,-80.189712,Tapas Restaurant
1,Arts & Entertainment District,25.799,-80.19,Plant Food + Wine Miami,25.800452,-80.192805,Restaurant
2,Arts & Entertainment District,25.799,-80.19,La Provence French Bakery,25.798969,-80.189127,French Restaurant
3,Arts & Entertainment District,25.799,-80.19,The Daily Creative Food Co. - Miami,25.795696,-80.188683,Sandwich Place
4,Arts & Entertainment District,25.799,-80.19,Mister O1 Extraordinary Pizza,25.799129,-80.194791,Pizza Place


In [158]:
miami_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arts & Entertainment District,15,15,15,15,15,15
Brickell,30,30,30,30,30,30
Coconut Grove,2,2,2,2,2,2
Coral Way,3,3,3,3,3,3
Design District,19,19,19,19,19,19
Downtown,50,50,50,50,50,50
Little Havana,13,13,13,13,13,13
Miami Beach,100,100,100,100,100,100
Midtown,43,43,43,43,43,43


In [159]:
print('There are {} uniques categories.'.format(len(miami_venues['Venue Category'].unique())))

There are 51 uniques categories.


In [160]:
# one hot encoding
miami_onehot = pd.get_dummies(miami_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
miami_onehot['Neighborhood'] = miami_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [miami_onehot.columns[-1]] + list(miami_onehot.columns[:-1])
miami_onehot = miami_onehot[fixed_columns]

miami_onehot.head()

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Eastern European Restaurant,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Peruvian Restaurant,Pizza Place,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant
0,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [161]:
miami_onehot.shape

(322, 52)

In [162]:
miami_grouped = miami_onehot.groupby('Neighborhood').mean().reset_index()
miami_grouped

,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Chinese Restaurant,Cuban Restaurant,Deli / Bodega,Eastern European Restaurant,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Greek Restaurant,Hot Dog Joint,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,New American Restaurant,Peruvian Restaurant,Pizza Place,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant
0,Arts & Entertainment District,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.066667,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.133333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.133333,0.00,0.000000,0.000000
1,Brickell,0.133333,0.000000,0.100000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.033333,0.033333,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.066667,0.100000,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.066667,0.033333,0.066667,0.033333,0.000000,0.00,0.033333,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.033333
2,Coconut Grove,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.000000,0.5,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Coral Way,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.333333,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Design District,0.105263,0.052632,0.000000,0.000000,0.00,0.00,0.052632,0.000000,0.000000,0.000000,0.000000,0.210526,0.052632,0.000000,0.000000,0.00,0.052632,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.052632,0.00,0.105263,0.052632,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,Downtown,0.040000,0.000000,0.000000,0.020000,0.00,0.02,0.000000,0.020000,0.040000,0.040000,0.000000,0.100000,0.020000,0.000000,0.000000,0.02,0.000000,0.000000,0.000000,0.020000,0.020000,0.0,0.000000,0.02,0.060000,0.0,0.000000,0.02,0.100000,0.000000,0.020000,0.020000,0.000000,0.000000,0.040000,0.080000,0.060000,0.060000,0.020000,0.080000,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000
6,Little Havana,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.153846,0.000000,0.076923,0.00,0.153846,0.076923,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.230769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
7,Miami Beach,0.050000,0.000000,0.020

### 

In [169]:
num_top_venues = 35

for hood in miami_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = miami_grouped[miami_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Restaurant','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    temp.drop(temp[temp['Frequency'] == 0.00].index, inplace = True) 
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arts & Entertainment District----
          Restaurant  Frequency
0        Pizza Place       0.20
1         Restaurant       0.20
2     Sandwich Place       0.13
3   Tapas Restaurant       0.13
4             Bistro       0.07
5     Breakfast Spot       0.07
6   Cuban Restaurant       0.07
7         Food Truck       0.07
8  French Restaurant       0.07


----Brickell----
                   Restaurant  Frequency
0         American Restaurant       0.13
1         Japanese Restaurant       0.10
2      Argentinian Restaurant       0.10
3                        Café       0.07
4                        Food       0.07
5          Italian Restaurant       0.07
6              Sandwich Place       0.07
7                  Restaurant       0.07
8                 Salad Place       0.03
9                  Steakhouse       0.03
10         Spanish Restaurant       0.03
11         Seafood Restaurant       0.03
12         Mexican Restaurant       0.03
13    New American Restaurant       0.03
14  Lati

In [170]:
for col in miami_grouped.columns: 
    print(col) 

Neighborhood
American Restaurant
Arepa Restaurant
Argentinian Restaurant
Asian Restaurant
BBQ Joint
Bagel Shop
Bakery
Bistro
Brazilian Restaurant
Breakfast Spot
Burger Joint
Café
Chinese Restaurant
Cuban Restaurant
Deli / Bodega
Eastern European Restaurant
Fast Food Restaurant
Food
Food Court
Food Truck
French Restaurant
Fried Chicken Joint
Gastropub
Gluten-free Restaurant
Greek Restaurant
Hot Dog Joint
Indian Restaurant
Indonesian Restaurant
Italian Restaurant
Japanese Restaurant
Latin American Restaurant
Mediterranean Restaurant
Mexican Restaurant
Middle Eastern Restaurant
New American Restaurant
Peruvian Restaurant
Pizza Place
Restaurant
Salad Place
Sandwich Place
Seafood Restaurant
South American Restaurant
Southern / Soul Food Restaurant
Spanish Restaurant
Steakhouse
Sushi Restaurant
Taco Place
Tapas Restaurant
Turkish Restaurant
Vegetarian / Vegan Restaurant
Venezuelan Restaurant
